In [1]:
from utils.df_handle import *

In [2]:
# day_ago = 2
datenow = datetime.now().strftime("%Y%m%d")
# datenow_day_ago = ( datetime.now()-timedelta(day_ago) ).strftime("%Y%m%d")
# param_1 = f"'{datenow_day_ago}'"
param_2 = f"'20161130'"
# param_4 = f"'20211027'"
param_3 = f"'{datenow}'"

In [3]:
if datetime.now().hour in {8,9,10,11,12,13,14,15,16,17,18,22}:
    print ("OK")

OK


In [4]:
# pr_AR_RawdataTrackingDebtConfirm_Renew 825

In [5]:
# query = f"EXEC [pr_AR_RawdataTrackingDebtConfirm_Renew] @RPTID=825"
# start=time.time()
# df1 = get_ms_df(sql=query)
# end=time.time()
# duration=end-start
# print(duration)

In [ ]:
# df1['']

In [17]:
df1.columns

Index(['Mã Chi Nhánh/Cty', 'Tên Chi Nhánh/Cty', 'Quản Lý MDS', 'Mã Nhân Viên',
       'Tên Nhân Viên', 'Vai Trò', 'Hình Thức Thanh Toán', 'Hạn Thanh Toán',
       'Ngày Nghiệp Vụ', 'Mã Đơn Hàng', 'Số Hóa Đơn', 'Ký Hiệu Hóa Đơn',
       'Ngày Đơn Hàng', 'Ngày', 'Tháng', 'Năm', 'Đơn Vị Giao Hàng',
       'Thời Gian Giao Hàng (HH:MM)', 'Thời Hạn Thanh Toán', 'Ngày Đến Hạn',
       'Mã Khách Hàng', 'Tên Khách Hàng', 'Mã Kênh', 'Mã Kênh Phụ', 'Khu Vực',
       'Số Nhà Và Tên Đường', 'Phường/Xã', 'Quận/Huyện', 'Tỉnh/Thành',
       'Người Liên Hệ', 'Số Điện Thoại', 'Số Điện Thoại (KH Thuế)',
       'Đầu Kỳ (ĐH)', 'Đầu Kỳ (Số Tiền)', 'Chốt Sổ (ĐH)', 'Chốt sổ (Số Tiền)',
       'Giao Thành Công (ĐH)', 'Giao Thành Công (Số Tiền)',
       'Hủy Hóa Đơn - Trả Hàng (ĐH)', 'Hủy Hóa Đơn - Trả Hàng (Số Tiền)',
       'Xác Nhận Thu Nợ (Số Tiền)', 'Lý Do Không Thu Nợ Được',
       'Tạo Bảng Kê (ĐH)', 'Tạo Bảng Kê (Số Tiền)', 'Xác Nhận TT Công Nợ (ĐH)',
       'Xác Nhận TT Công Nợ (Số Tiền)', 'Dư Nợ Cuối 

In [36]:
ctr1 = df1['Phụ Trách Nợ'] == 'MDS'

In [37]:
ctr1

0         False
1         False
2         False
3         False
4         False
          ...  
269762     True
269763     True
269764     True
269765    False
269766    False
Name: Phụ Trách Nợ, Length: 269767, dtype: bool

In [38]:
df2 = df1[ctr1]

In [21]:
df2['Dư Nợ Cuối Kỳ'].sum()

5260954970.0

In [ ]:
# ctr2 = df1['Thời Hạn Thanh Toán'].isin(['Thu tiền ngay không có VP PN','Thu tiền ngay có VP PN','Gối 1 Đơn Hàng (trong 30 ngày)'])

In [6]:
query = f"EXEC [pr_AR_TrackingDebtConfirm] @Fromdate={param_2}, @Todate={param_3}"
start=time.time()
df1 = get_ms_df(sql=query, parse_dates={"DateOfOrder": {"dayfirst": True}, "DueDate": {"dayfirst": True}, "OrderDate": {"dayfirst": True}})
# df1.to_csv('doanhthutienmat_data1.csv', index = False)
end=time.time()
duration=end-start
print(duration)

27.77555513381958


In [7]:
# ctr1 = df1['DebtInCharge'] == 'MDS'

In [8]:
# df2 = df1[ctr1]

In [9]:
# df2.columns

In [98]:
# Handle Hanh input

In [99]:
HANH_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTbIG2vkBwwtWCfZL7m5IE0Wq0F9UwZ7-KhzPgBgLObMb2C43ANH79rgGDeyQXa4C48thAbTCavUxuI/pub?gid=0&single=true&output=csv'
HANH_csv = pd.read_csv(HANH_URL)
HANH_csv['madh_cn'] = HANH_csv['MA CN'] + HANH_csv['MA DH']
df1['madh_cn'] = df1.BranchID + df1.OrderNbr

In [106]:
ctr1 = HANH_csv['Due Date'].notna()
df1['DueDate2'] = df1['DueDate']
df1['DueDate'] = df1['madh_cn'].map(df_to_dict( HANH_csv[['madh_cn','Due Date']][ctr1] ))
df1['DueDate']=pd.to_datetime(df1['DueDate'], dayfirst=True)
df1['DueDate'].fillna(df1['DueDate2'], inplace=True)
drop_cols(df1, ['DueDate2'])

In [121]:
x = df1['madh_cn']
ctr2 = HANH_csv['THTT'].notna()
set2 = set(HANH_csv[ctr2]['madh_cn'].unique())
dict2 = df_to_dict( HANH_csv[['madh_cn','THTT']][ctr2] )
# THTT = Terms
df1['Terms'] = np.where(x.isin(set2), x.map( dict2 ), df1['Terms'] )

In [135]:
ctr3 = HANH_csv['HAN THANH TOAN'].notna()
set3 = set(HANH_csv[ctr3]['madh_cn'].unique())
dict3 = df_to_dict( HANH_csv[['madh_cn','HAN THANH TOAN']][ctr3] )
# Hanh thanh toan = TermType
df1['TermsType'] = np.where(x.isin(set3), x.map( dict3 ), df1['TermsType'] )

In [136]:
ctr3 = HANH_csv['HTTH'].notna()
set3 = set(HANH_csv[ctr3]['madh_cn'].unique())
dict3 = df_to_dict( HANH_csv[['madh_cn','HTTH']][ctr3] )
# HTTH = PaymentsForm
df1['PaymentsForm'] = np.where(x.isin(set3), x.map( dict3 ), df1['PaymentsForm'] )

In [137]:
ctr3 = HANH_csv['BEN PHU TRACH NO'].notna()
set3 = set(HANH_csv[ctr3]['madh_cn'].unique())
dict3 = df_to_dict( HANH_csv[['madh_cn','BEN PHU TRACH NO']][ctr3] )
# Ben Phu Trach No = DebtInCharge
df1['DebtInCharge'] = np.where(x.isin(set3), x.map( dict3 ), df1['DebtInCharge'] )

In [138]:
ctr3 = HANH_csv['MA NV'].notna()
set3 = set(HANH_csv[ctr3]['madh_cn'].unique())
dict3 = df_to_dict( HANH_csv[['madh_cn','MA NV']][ctr3] )
# MA NV = SlsperID
df1['SlsperID'] = np.where(x.isin(set3), x.map( dict3 ), df1['SlsperID'] )

In [140]:
drop_cols(df1, ['madh_cn'])
del(HANH_csv)

In [7]:
# df_filter(df1, BranchID='MR0001', OrderNbr='DH112021-01987')

In [51]:
# df1.columns = [x.lower() for x in df1.columns]

In [53]:
# df1[checkdup(df1, 2, ['ordernbr', 'branchid', 'dateoforder', 'duedate', 'orderdate'])]

,branchid,branchname,supname,asmname,rsmname,slsperid,slspername,position,debtincharge,paymentsform,...,deliveredorder,deliveredorderamt,countreturnord,returnordamt,confirmamt,reasonnopay,countdebtconfirm,debtconfirmamt,countdebtconfirmrelease,debtconfirmamtrelease


In [16]:
# df_filter(df1, OrderNbr='DH062021-00112', BranchID='MR0012').to_clipboard()

In [11]:
# df1 = df_filter(df1,OrderNbr='DH062021-00112',BranchID='MR0012')

In [12]:
# df1.to_clipboard()

In [23]:
# df1.to_clipboard()

In [24]:
# query = f"SELECT BranchID, OrderNbr, InvcNbr, InvcNote, DateOfOrder, DueDate, CustID FROM AR_RawdataTrackingDebtConfirm_BL WHERE Terms=N'Gối 1 Đơn Hàng (trong 30 ngày)'"
# df1 = get_ms_df(sql=query, parse_dates={"DateOfOrder": {"dayfirst": True}, "DueDate": {"dayfirst": True}})
# df1a=df1.copy()

In [25]:
# df1a.columns

In [118]:
# df1a = df_filter(df1a,CustID='TN80E088')

In [119]:
# df1a.shape
# df1a.sort_values(['DateOfOrder', 'OrderNbr'], axis=0, ascending = (True, True), inplace=True)

In [120]:
# df1a.DueDate = df1a.DateOfOrder+timedelta(30)

In [121]:
# df1a.to_clipboard()

In [122]:
# df1b = df1a[['BranchID','CustID', 'OrderNbr']].copy()
# df1b.drop_duplicates(keep='first', inplace=True)
# df1b.to_clipboard()
# df1b['cum_count'] = df1b.groupby(['BranchID','CustID']).cumcount()
# df1a = df1a.merge(df1b, how='left', on=['BranchID','CustID', 'OrderNbr'])
# df1a['cum_count_plus1'] = df1a['cum_count']+1

# # df1a.to_clipboard()
# df1b = df1a[['BranchID', 'CustID', 'cum_count', 'OrderNbr', 'DateOfOrder']].drop_duplicates(keep='first')
# df1b.to_csv('df1b.csv')
# df1a = df1a.merge(df1b.add_prefix('y_'), how='left', left_on=['BranchID', 'CustID', 'cum_count_plus1'], right_on=['y_BranchID','y_CustID','y_cum_count'])
# # df1a.to_clipboard()
# df1a.y_DateOfOrder.fillna(df1a.DueDate, inplace=True)
# df1a.DueDate = np.where(df1a.DueDate>df1a.y_DateOfOrder, df1a.y_DateOfOrder, df1a.DateOfOrder+timedelta(30))
# df1a.to_clipboard()

In [ ]:
datenow = datetime.now().strftime("%Y%m%d")
print(datenow)

In [ ]:
datenow = datetime.now().strftime("%Y%m%d")
query = f"EXEC pr_AR_RawdataTrackingDebtConfirm_BL 2,'{datenow}','{datenow}','';"
commit_ms_sql(query)
query = f"SELECT BranchID, OrderNbr, DateOfOrder, CustID, sum(DebtBalance) as DebtBalance FROM AR_RawdataTrackingDebtConfirm_BL WHERE Terms=N'Gối 1 Đơn Hàng (trong 30 ngày)' GROUP BY BranchID, OrderNbr, DateOfOrder, CustID"
df1 = get_ms_df(sql=query, parse_dates={"DateOfOrder": {"dayfirst": True}, "DueDate": {"dayfirst": True}})
df1['DueDate'] = df1.DateOfOrder+timedelta(30)
df1.sort_values(['DateOfOrder', 'OrderNbr'], axis=0, ascending = (True, True), inplace=True)
# CASE 1
ctr1 = df1['DebtBalance'] > 0
df1a=df1[ctr1].copy()
# Convert DateOfOrder2, next value same or continuous
df1a['group'] = df1a.groupby(['BranchID','CustID'])['DateOfOrder'].diff().gt(pd.to_timedelta('1d'))
df1a['group'] = df1a.groupby(['BranchID','CustID'])['group'].cumsum()
df1a2 = df1a.groupby(['BranchID','CustID', 'group'], as_index=False).agg({'DateOfOrder': np.max})
df1a = df1a.merge(df1a2, how = 'left', on=['BranchID','CustID','group'], suffixes=('_ori', ''), validate="m:1")
drop_cols(df1a, ['group'])
del(df1a2)
# Calculate based on DateOfOrder2
df1b = df1a[['BranchID','CustID', 'DateOfOrder']].copy()
df1b.drop_duplicates(keep='first', inplace=True)
df1b['cum_count'] = df1b.groupby(['BranchID','CustID']).cumcount()
df1a = df1a.merge(df1b, how='left', on=['BranchID','CustID', 'DateOfOrder'])
df1a['cum_count_plus1'] = df1a['cum_count']+1
df1b = df1a[['BranchID', 'CustID', 'cum_count', 'OrderNbr', 'DateOfOrder']].drop_duplicates(keep='first')
df1a = df1a.merge(df1b.add_prefix('y_'), how='left', left_on=['BranchID', 'CustID', 'cum_count_plus1'], right_on=['y_BranchID','y_CustID','y_cum_count'])
df1a.y_DateOfOrder.fillna(df1a.DueDate, inplace=True)
df1a.DueDate = np.where(df1a.DueDate>df1a.y_DateOfOrder, df1a.y_DateOfOrder, df1a.DueDate)
#
drop_cols(df1a, ['DateOfOrder'])
df1a.rename(columns={"DateOfOrder_ori": "DateOfOrder"}, inplace=True)
#
cols_list = df1.columns
df1a = df1a[cols_list]
df1a.drop_duplicates(keep='first', inplace=True)
# CASE 2
df1c=df1[~ctr1].copy()
df1c.drop_duplicates(keep='first', inplace=True)
# Merge
df1d = union_all([df1a, df1c])
drop_cols(df1d, ['DebtBalance'])
del(df1)
update_sql = \
"""
UPDATE d SET d.DueDate=t.DueDate
FROM  dbo.AR_RawdataTrackingDebtConfirm_BL d
INNER JOIN #tempt1 t ON t.BranchID = d.BranchID AND t.OrderNbr = d.OrderNbr
"""
insert_to_ms_and_update(df1d, '#tempt1', update_sql)

In [5]:
# drop_sql = \
# """
# DROP TABLE IF EXISTS #tempt1;
# """
# commit_ms_sql(drop_sql)

In [6]:
# get_ms_csv("select * from #tempt", "tempt.csv")

In [7]:
# df1a.to_csv('output.csv')

In [8]:
# Update 29/10
# ctr1 = df1.Terms == 'Gối 1 Đơn Hàng (trong 30 ngày)'
# df1a=df1[ctr1].copy()
# df1a.sort_values('DateOfOrder', axis=0, ascending = True, inplace=True)
# df1b = df1a[['BranchID','CustID','OrderNbr']].copy()
# df1b.drop_duplicates(keep='first', inplace=True)
# df1b['cum_count'] = df1b.groupby(['BranchID','CustID']).cumcount()
# df1a = df1a.merge(df1b, how='left', on=['BranchID','CustID', 'OrderNbr'])
# df1a['cum_count_plus1'] = df1a['cum_count']+1
# df1a = df1a.merge(df1a[['BranchID', 'CustID', 'cum_count', 'OrderNbr', 'DateOfOrder']].drop_duplicates(keep='first').add_prefix('y_'), how='left', left_on=['BranchID', 'CustID', 'cum_count_plus1', 'OrderNbr'], right_on=['y_BranchID','y_CustID','y_cum_count', 'y_OrderNbr'])
# df1a.y_DateOfOrder.fillna(df1a.DueDate, inplace=True)
# df1a.DueDate = np.where(df1a.DueDate>df1a.y_DateOfOrder, df1a.y_DateOfOrder, df1a.DueDate)
# cols_list = df1.columns
# df1a = df1a[cols_list]
# df1 = df1[~ctr1].append(df1a)
# del(df1a)
# del(df1b)

In [9]:
# df1.shape

In [10]:
# df1a[['BranchID', 'CustID', 'cum_count', 'OrderNbr', 'DateOfOrder']].drop_duplicates(keep='first')

In [11]:
# checkdup(df2,1,['BranchID', 'CustID', 'DueDate', 'OrderNbr']).sum()

In [12]:
# sys.getsizeof(df2)/(1024*1024)

In [13]:
# df_filter(df1, OrderNbr='DH102021-01793', BranchID='MR0010').to_clipboard()

In [14]:
# df1.dtypes

In [15]:
# df_filter(df1, BranchID='MR0010', OrderNbr='DH012021-06151').to_clipboard()

In [16]:
# df1.columns

In [139]:
# df1[checkdup(df1, 2, ['BranchID','OrderDate','OrderNbr','DateOfOrder','DueDate'])]

In [5]:
# ctr1 = df1.DateOfOrder >= datetime(2021, 8,1)
# ctr2 = df1.DateOfOrder <= datetime(2021, 8,31)

In [6]:
# df2 = df1[ctr1&ctr2]

In [7]:
# df3 = df2[['DateOfOrder', 'OrderDate']].copy()

In [8]:
# df3.head()

In [9]:
# df3.OrderDate.nbytes/(1024*1024)

In [10]:
# df_filter(df1, CustID='TN80E088').to_clipboard()

In [11]:
# df1.to_csv('csv1.csv')

In [35]:
# df_nhansu = get_ps_df("select * from d_nhan_su")
# df_nhansu['qlkhuvuc'] = df_nhansu['qlkhuvuc'].str.strip()
# df_nhansu[['manvcrscrss','qlkhuvuc']].copy().drop_duplicates(inplace=True)
# df1a
# df1a.head()
# df_nhansu_asm = dropdup(df_nhansu[['manvcrscrss','qlkhuvuc']], 1)
# df_nhansu_asm.head()
# nhansu_asm_dict = df_to_dict(df_nhansu_asm)

In [10]:
df_nhansu = get_ps_df("select * from d_nhan_su")
df_nhansu['qlkhuvuc'] = df_nhansu['qlkhuvuc'].str.strip()
df_nhansu_asm = dropdup(df_nhansu[['manvcrscrss','qlkhuvuc']], 1)
nhansu_asm_dict = df_to_dict(df_nhansu_asm)
df_phu_trach_no_cs_theo_tinh = get_ps_df("SELECT * from d_phu_trach_no_cs_theo_tinh")
# df_phu_trach_no_cs_theo_tinh.columns
phu_trach_no_cs_theo_tinh_dict = df_to_dict(df_phu_trach_no_cs_theo_tinh)
df1['ASMName'] = np.where(df1['DebtInCharge']=='MDS', df1['SlsperID'].map(nhansu_asm_dict).fillna('Lương Trịnh Thắng (KN)'), df1['State'].map(phu_trach_no_cs_theo_tinh_dict))
# df1['InChargeName'] =  np.where(df1['DebtInCharge']=='MDS', df1['SlsperName'], df1['ASMName'])
# list_cs = df_phu_trach_no_cs_theo_refcustid['refcustid'].to_list()

In [11]:
# ((df1['RefCustId'].isin(list_cs)) & (df1['DebtInCharge']=='CS'))

In [12]:
ma_kh_cu_dict = get_ms_df("SELECT CustId, RefCustID from AR_Customer").set_index('CustId').to_dict()['RefCustID']
df1['RefCustId'] = df1['CustID'].map(ma_kh_cu_dict)
df_phu_trach_no_cs_theo_refcustid = get_ps_df("select * from d_phu_trach_no_cs_theo_refcustid")
phu_trach_no_cs_theo_refcustid_dict = get_ps_df("select * from d_phu_trach_no_cs_theo_refcustid").set_index('refcustid').to_dict()['inchargename']
list_cs = df_phu_trach_no_cs_theo_refcustid['refcustid'].to_list()
df1['ASMName_CS'] = df1['RefCustId'].map(phu_trach_no_cs_theo_refcustid_dict)
df1['ASMName'] = np.where( ((df1['RefCustId'].isin(list_cs)) & (df1['DebtInCharge']=='CS')), df1['ASMName_CS'], df1['ASMName'])

In [13]:
df_nhansu_sup = dropdup(df_nhansu[['manvcrscrss','quanlytructiep']], 1)

In [14]:
df_nhansu_sup['quanlytructiep'] = df_nhansu_sup['quanlytructiep'].str.strip()

In [15]:
# df_to_dict(df_nhansu_sup)

In [16]:
df1['SupName'] = df1['SupName'].map( df_to_dict(df_nhansu_sup) ).fillna(df1['ASMName'])

In [17]:
df_nhansu_rsm = dropdup(df_nhansu[['manvcrscrss','qlvung']], 1)

In [18]:
df_nhansu_rsm['qlvung'] = df_nhansu_rsm['qlvung'].str.strip()

In [19]:
df1['SupName'] = df1['SupName'].map( df_to_dict(df_nhansu_sup) ).fillna(df1['ASMName'])

In [20]:
df1['RSMName'] = df1['RSMName'].map( df_to_dict(df_nhansu_rsm) ).fillna(df1['ASMName'])

In [21]:
# ma_kh_cu_dict = get_ms_df("SELECT CustId, RefCustID from AR_Customer").set_index('CustId').to_dict()['RefCustID']
# df1['RefCustId'] = df1['CustID'].map(ma_kh_cu_dict)

In [22]:
# df_phu_trach_no_cs_theo_refcustid = get_ps_df("select * from phu_trach_no_cs_theo_refcustid")
# phu_trach_no_cs_theo_refcustid_dict = get_ps_df("select * from phu_trach_no_cs_theo_refcustid").set_index('refcustid').to_dict()['inchargename']
# list_cs = df_phu_trach_no_cs_theo_refcustid['refcustid'].to_list()
# df1['ASMName_CS'] = df1['RefCustId'].map(phu_trach_no_cs_theo_refcustid_dict)
# df1['ASMName'] = np.where(df1['RefCustId'].isin(list_cs), df1['ASMName_CS'], df1['ASMName'])

In [23]:
df1['InChargeName'] =  np.where(df1['DebtInCharge']=='MDS', df1['SlsperName'], df1['ASMName'])

In [24]:
# vc(df1, 'ASMName')

In [25]:
df_vptt = get_ps_df("select * from d_vptt")
df1 = df1.merge(df_vptt, how = 'left', left_on='State', right_on='tinh', suffixes=('_left', '_right'), validate="m:1")

In [26]:
df_mkv_viet_tat = get_ps_df("select * from d_mkv_viet_tat").set_index('tenkhuvuc')

In [27]:
khuvuc_dict = df_mkv_viet_tat.to_dict()['khuvuc']
df1['Territory'] = df1['Territory'].map(khuvuc_dict)

In [28]:
df1['Position'] = np.where(df1['DebtInCharge']=="CS", "CS", df1['Position'])

In [29]:
# df1.dtypes

In [30]:
# df1['Action_MY'] = df1['OrderDate'].dt.month.astype('str')+df1['OrderDate'].dt.year.astype('str')

In [31]:
# df1['Order_MY'] = df1['DateOfOrder'].dt.month.astype('str')+df1['DateOfOrder'].dt.year.astype('str')

In [32]:
# df1['ReturnOrdAmt_InMonth'] = np.where( df1['Order_MY']==df1['Action_MY'], df1['ReturnOrdAmt'], 0)
# df1['DebConfirmAmtRelease_InMonth'] = np.where( df1['Order_MY']==df1['Action_MY'], df1['DebConfirmAmtRelease'], 0)

In [33]:
df1[checkdup(df1, 2, ['BranchID','OrderDate','OrderNbr','DateOfOrder','DueDate'])]

,BranchID,BranchName,SupName,ASMName,RSMName,SlsperID,SlsperName,Position,DebtInCharge,PaymentsForm,...,ReasonNoPay,CountDebtConfirm,DebtConfirmAmt,CountDebtConfirmRelease,DebtConfirmAmtRelease,RefCustId,ASMName_CS,InChargeName,tinh,vptt


In [34]:
groupbylist = [
    'OrderNbr', 
    'BranchID', 
    'Position',
    'SlsperID', 
    'SupName', 
    'ASMName', 
    'RSMName', 
    'DateOfOrder', 
    'DueDate',
    'CustID',
    'RefCustId',
    'CustName',
    'SlsperName',
    'InChargeName',
    'DebtInCharge',
    'Terms', 
    'PaymentsForm',
    'TermsType',
    'Territory', 
    'State', 
    'vptt', 
    'DeliveryUnit',
    'Channels',
    'SubChannel'
    ]

In [35]:
aggregate_dict = {
'OrderDate': np.max,
#Group by tien
'OpeningOrderAmt':np.sum,
'OrdAmtRelease':np.sum,
'DeliveredOrderAmt':np.sum,
'ReturnOrdAmt': np.sum,
'DebtConfirmAmt': np.sum,
'DebtConfirmAmtRelease': np.sum,
# Huy & Xac Nhan In Month
# 'ReturnOrdAmt_InMonth': np.sum,
# 'DebConfirmAmtRelease_InMonth': np.sum,

#DonHang8
'CountOpeningOrder':np.sum,
'CountOrdRelease': np.sum,
'DeliveredOrder':np.sum,
'CountReturnOrd': np.sum,
'CountDebtConfirm': np.sum,
'CountDebtConfirmRelease':np.sum
}

In [51]:
df2 = pivot(df1, groupbylist, aggregate_dict)

In [52]:
# df2.columns = [x.lower() for x in df2.columns]

In [53]:
# df2[checkdup(df2, 2, ['BranchID','OrderDate','OrderNbr','DateOfOrder','DueDate'])]

In [54]:
# df2.columns

In [55]:
rename_dict ={
# Doanh So
'OpeningOrderAmt': 'tiennodauky',
'OrdAmtRelease':'tienchotso',
'DeliveredOrderAmt':'tiengiaothanhcong',
'ReturnOrdAmt':'tienhuydon',
'DebtConfirmAmt':'tienlenbangke',
'DebtConfirmAmtRelease':'tienthuquyxacnhan',
# Huy & Xac Nhan In Month
# 'ReturnOrdAmt_InMonth': 'tienhuydon_inmonth',
# 'DebConfirmAmtRelease_InMonth': 'tienthuquyxacnhan_inmonth',
# Don Hang
'CountOpeningOrder': 'dondauky',
'CountOrdRelease':'donchotso',
'DeliveredOrder':'dongiaothanhcong',
'CountReturnOrd':'donhuy',
'CountDebtConfirm':'donlenbangke',
'CountDebtConfirmRelease':'donthuquyxacnhan'
}
df2.rename(columns=rename_dict, inplace=True)

In [56]:
# df2.to_clipboard()

In [57]:
# df2.columns

In [58]:
df2.columns

Index(['OrderNbr', 'BranchID', 'Position', 'SlsperID', 'SupName', 'ASMName',
       'RSMName', 'DateOfOrder', 'DueDate', 'CustID', 'RefCustId', 'CustName',
       'SlsperName', 'InChargeName', 'DebtInCharge', 'Terms', 'PaymentsForm',
       'TermsType', 'Territory', 'State', 'vptt', 'DeliveryUnit', 'Channels',
       'SubChannel', 'OrderDate', 'tiennodauky', 'tienchotso',
       'tiengiaothanhcong', 'tienhuydon', 'tienlenbangke', 'tienthuquyxacnhan',
       'dondauky', 'donchotso', 'dongiaothanhcong', 'donhuy', 'donlenbangke',
       'donthuquyxacnhan'],
      dtype='object')

In [59]:
# update 04/11
ctr1 = df2['DeliveryUnit']=='Nhà vận chuyển'
ctr2 = df2['donhuy']==0
df2['dongiaothanhcong'] = np.where(ctr1&ctr2, 1, df2['dongiaothanhcong'])

In [60]:
# update 05/11
ctr1 = df2['DeliveryUnit']=='Nhà vận chuyển'
ctr2 = df2['tienhuydon']==0
df2['tiengiaothanhcong'] = np.where(ctr1&ctr2, df2['tienchotso'], df2['tiengiaothanhcong'])

In [61]:
df2['tiennocongty'] = df2['tiennodauky'] + df2['tienchotso'].values - df2['tienhuydon'].values - df2['tienthuquyxacnhan'].values
# df2['tiennocongty'] = np.where(df2['chitietnocongty']>0, df2['chitietnocongty'].values, 0)
# df2['chitietdonnocongty'] = df2['dondauky'] + df2['donchotso'] - df2['donthuquyxacnhan'] - df2['donhuy']
# df2['donnocongty_old'] = df2.apply(lambda x: x['chitietdonnocongty'] if x['chitietdonnocongty']>0 else 0, axis=1)
df2['donnocongty'] = np.where(df2['tiennocongty'].values > 0, 1, 0)

df2['donchuagiao'] = df2['donchotso'] - df2['dongiaothanhcong'] - df2['donhuy']
df2['tiendonchuagiao'] = df2['tienchotso'] - df2['tiengiaothanhcong'] - df2['tienhuydon']

# date_now_at_midnight = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
# df2['OverDueDate'] = np.where(df2['DueDate'].values <= np.datetime64(date_now_at_midnight), True, False)
# # df2['OverDueDate'].value_counts()
# df2['nodenhan'] = np.where(df2['OverDueDate'].values, df2['tiennocongty'].values, 0)
# df2['nochuadenhan'] = np.where(df2['OverDueDate'].values != True, df2['tiennocongty'].values, 0)

In [62]:
# df2.to_clipboard()

In [63]:
# df2[checkdup(df2, 2, ['BranchID','OrderNbr'])]

In [64]:
# ctr1 = df2['DeliveryUnit']=='Nhà vận chuyển'

In [65]:
# df3 = df2[ctr1]

In [67]:
df2.duplicated().sum()

0

In [44]:
df2.columns

Index(['ordernbr', 'branchid', 'position', 'slsperid', 'supname', 'asmname',
       'rsmname', 'dateoforder', 'duedate', 'custid', 'refcustid', 'custname',
       'slspername', 'inchargename', 'debtincharge', 'terms', 'paymentsform',
       'termstype', 'territory', 'state', 'vptt', 'deliveryunit', 'channels',
       'subchannel', 'orderdate', 'openingorderamt', 'ordamtrelease',
       'deliveredorderamt', 'returnordamt', 'debtconfirmamt',
       'debtconfirmamtrelease', 'countopeningorder', 'countordrelease',
       'deliveredorder', 'countreturnord', 'countdebtconfirm',
       'countdebtconfirmrelease'],
      dtype='object')

In [45]:
# df3.tiendonchuagiao.sum()

In [46]:
df2.columns = [x.lower() for x in df2.columns]

In [47]:
# df2[checkdup(df2, 2, ['ordernbr', 'branchid', 'dateoforder', 'duedate'])].to_clipboard()

In [48]:
df2.columns = [x.lower() for x in df2.columns]
trangthaigiaohangdict = {'C':'Đã giao hàng', 'H':'Chưa Xác Nhận', 'D':'KH Không Nhận', 'A':'Đã Xác Nhận', 'R':'Từ Chối Giao Hàng', 'E':'Không Tiếp Tục Giao Hàng'}
# get_ms_csv(DELI, 'DELI.csv')
DELI=pd.read_csv('DELI.csv', usecols=['BranchID', 'OrderNbr', 'Status'])
DELI['BranchID_OrderNbr'] = DELI['BranchID']+DELI['OrderNbr']
# DELI.columns
DELI = DELI[['BranchID_OrderNbr','Status']]
# df2.columns
df2['trangthaigiaohang'] = (df2['branchid']+df2['ordernbr']).map(df_to_dict(DELI)).map(trangthaigiaohangdict)
# vc(df2,'deliveryunit')

In [50]:
# df2=df2[['ordernbr', 'branchid', 'dateoforder', 'duedate', 'tienhuydon_inmonth','tienthuquyxacnhan_inmonth']]

In [51]:
primary_keys=['ordernbr', 'branchid', 'dateoforder', 'duedate']

In [52]:
df2['inserted_at'] = datetime.now()

In [54]:
dropdup(df2,1,['ordernbr', 'branchid', 'dateoforder', 'duedate'])

,ordernbr,branchid,position,slsperid,supname,asmname,rsmname,dateoforder,duedate,custid,...,tienlenbangke,tienthuquyxacnhan,dondauky,donchotso,dongiaothanhcong,donhuy,donlenbangke,donthuquyxacnhan,trangthaigiaohang,inserted_at
0,DH012017-12644,MR0003,CS,MR2168,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2017-01-18,2017-04-18,TD42I014,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
1,DH012017-13345,MR0001,CS,MR0456,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2017-01-18,2017-04-18,N02209270,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
2,DH012017-13346,MR0001,CS,MR0456,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2017-01-18,2017-04-18,N02209270,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
3,DH012017-13360,MR0001,CS,MR0456,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2017-01-18,2017-04-18,N02209270,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
4,DH012019-05622,MR0003,CS,MR2168,Lê Thị Mai Phương,Lê Thị Mai Phương,Lê Thị Mai Phương,2019-01-10,2019-04-10,TD30I028,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106596,DH122020-39619,MR0013,CS,MR1768,Đinh Thị Ngọc Mẫn,Đinh Thị Ngọc Mẫn,Đinh Thị Ngọc Mẫn,2020-12-30,2021-03-30,N0110332,...,7400000.0,7400000.0,1,0,0,0,1,1,NaN,2021-11-11 15:28:41.303347
106597,DH122020-39846,MR0010,CS,MR2000,Lê Thị Mai Phương,Lê Thị Mai Phương,Lê Thị Mai Phương,2020-12-30,2021-03-30,HH04E001,...,3700001.0,3700001.0,1,0,0,0,1,1,NaN,2021-11-11 15:28:41.303347
106598,DH122020-39876,MR0001,CS,MR2081,Lương Tấn Khả,Lương Tấn Khả,Lương Tấn Khả,2020-12-30,2021-03-30,TN90I018,...,0.0,0.0,1,0,0,0,0,0,NaN,2021-11-11 15:28:41.303347
106599,DH122020-40762,MR0001,CS,MR0246,Phạm Thị Cẩm Tú,Phạm Thị Cẩm Tú,Phạm Thị Cẩm Tú,2020-12-31,2021-03-31,M100902,...,1920000.0,1920000.0,1,0,0,0,1,1,NaN,2021-11-11 15:28:41.303347


In [56]:
df2[checkdup(df2, 2, ['ordernbr', 'branchid', 'dateoforder', 'duedate'])]

,ordernbr,branchid,position,slsperid,supname,asmname,rsmname,dateoforder,duedate,custid,...,tienlenbangke,tienthuquyxacnhan,dondauky,donchotso,dongiaothanhcong,donhuy,donlenbangke,donthuquyxacnhan,trangthaigiaohang,inserted_at
105655,DH112021-01987,MR0001,CS,MR2081,Lê Thị Mai Phương,Lê Thị Mai Phương,Lê Thị Mai Phương,2021-11-10,2022-02-08,TN90I048,...,0.0,0.0,0,1,0,0,0,0,NaN,2021-11-11 15:28:41.303347
105656,DH112021-01987,MR0001,CS,MR2081,Lê Thị Mai Phương,Lê Thị Mai Phương,Lê Thị Mai Phương,2021-11-10,2022-02-08,TN90I048,...,0.0,0.0,0,0,1,0,0,0,NaN,2021-11-11 15:28:41.303347


In [60]:
# rows = [tuple(x) for x in df2.to_numpy()]

In [61]:
# from datetime import datetime
# import pandas as pd
# import numpy as np
# import psycopg2.extras as extras
# from psycopg2 import OperationalError, errorcodes, errors
# import psycopg2
# from utils.df_handle import *

In [62]:
# conn = get_pg2_conn()

In [64]:
df2.to_csv('df2.csv')

In [ ]:
# TRUNCATE TABLE invoices CASCADE;

In [ ]:
commit_psql("truncate table f_tracking_debt cascade;")

In [49]:
execute_values_upsert(df2, 'f_tracking_debt', pk=primary_keys)

INSERT INTO f_tracking_debt(ordernbr,branchid,position,slsperid,supname,asmname,rsmname,dateoforder,duedate,custid,refcustid,custname,slspername,inchargename,debtincharge,terms,paymentsform,termstype,territory,state,vptt,deliveryunit,channels,subchannel,orderdate,tiennodauky,tienchotso,tiengiaothanhcong,tienhuydon,tienlenbangke,tienthuquyxacnhan,dondauky,donchotso,dongiaothanhcong,donhuy,donlenbangke,donthuquyxacnhan,tiennocongty,donnocongty,donchuagiao,tiendonchuagiao,trangthaigiaohang,inserted_at) VALUES %s ON CONFLICT (ordernbr, branchid, dateoforder, duedate) DO UPDATE SET position = excluded.position, slsperid = excluded.slsperid, supname = excluded.supname, asmname = excluded.asmname, rsmname = excluded.rsmname, custid = excluded.custid, refcustid = excluded.refcustid, custname = excluded.custname, slspername = excluded.slspername, inchargename = excluded.inchargename, debtincharge = excluded.debtincharge, terms = excluded.terms, paymentsform = excluded.paymentsform, termstype = 

In [46]:
# insert_rows_psql(table='f_tracking_debt', rows=rows, replace=True, target_fields=[x.lower() for x in df2.columns], replace_index=primary_keys)

Loaded %s rows into %s so far 1000 f_tracking_debt
Loaded %s rows into %s so far 2000 f_tracking_debt
Loaded %s rows into %s so far 3000 f_tracking_debt
Loaded %s rows into %s so far 4000 f_tracking_debt
Loaded %s rows into %s so far 5000 f_tracking_debt
Loaded %s rows into %s so far 6000 f_tracking_debt
Loaded %s rows into %s so far 7000 f_tracking_debt
Loaded %s rows into %s so far 8000 f_tracking_debt
Loaded %s rows into %s so far 9000 f_tracking_debt
Loaded %s rows into %s so far 10000 f_tracking_debt
Loaded %s rows into %s so far 11000 f_tracking_debt
Loaded %s rows into %s so far 12000 f_tracking_debt
Loaded %s rows into %s so far 13000 f_tracking_debt
Loaded %s rows into %s so far 14000 f_tracking_debt
Loaded %s rows into %s so far 15000 f_tracking_debt
Loaded %s rows into %s so far 16000 f_tracking_debt
Loaded %s rows into %s so far 17000 f_tracking_debt
Loaded %s rows into %s so far 18000 f_tracking_debt
Loaded %s rows into %s so far 19000 f_tracking_debt
Loaded %s rows into %

In [40]:
# insert_df_to_postgres(tbl_name="f_tracking_debt", df=df2, primary_keys=primary_keys)

In [41]:
# df2.to_clipboard()